# Demo: NO₃ Emulator (Keras, synthetic data)
This is a **demonstration** notebook to mimic your emulator pipeline. It:
1. Generates synthetic data with your feature names
2. Trains a small Keras MLP with **EarlyStopping** (similar flavour to AutoKeras)
3. Reports metrics and compares to a LinearRegression baseline

> Note: This is **self-contained** for review. Replace the model training with your
exported AutoKeras model when available (e.g., `model = tf.keras.models.load_model(...)`).

In [ ]:
import numpy as np, pandas as pd

import numpy as np
import pandas as pd

# For reproducibility
rng = np.random.default_rng(42)

# Synthetic data size
N = 5000  # keep this small enough for quick notebooks

# Feature names match your training_order (auto.py)
feature_names = [
    'lon','lat','WCDepth','chl_interp','pft_interp','sstost_interp',
    'temp_interp','d2m_interp','t2m_interp','sp_interp','tp_interp',
    'u10_interp','v10_interp','ssrdc_interp','phyc_interp','pp_interp',
    'closest_rorunoff','closest_rono3','closest_ronh4','closest_roo',
    'closest_rop','closest_rosio2','bathy_interp','month','day'
]

# Scientifically plausible ranges (very rough, shelf-focused)
lon = rng.uniform(-12, 10, N)           # degrees_east (AMM7-ish)
lat = rng.uniform(48, 62, N)            # degrees_north
WCDepth = rng.uniform(0, 5, N)          # m (surface sample depth)
chl = rng.uniform(0.05, 10, N)          # mg m^-3
pft = rng.uniform(0, 5, N)              # arbitrary unit (component of chl)
sstost = rng.uniform(6, 20, N)          # degC (OSTIA SST)
temp = rng.uniform(5, 18, N)            # degC (CMEMS thetao surface)
d2m = rng.uniform(270, 295, N)          # K
t2m = rng.uniform(270, 298, N)          # K
sp = rng.uniform(9.9e4, 1.03e5, N)      # Pa
tp = rng.exponential(2e-3, N)           # m/day precip (ERA5 daily)
u10 = rng.normal(0, 6, N)               # m/s
v10 = rng.normal(0, 6, N)               # m/s
ssrdc = rng.uniform(20, 280, N)         # W m^-2 (clear-sky daily avg approx)
phyc = rng.uniform(0.05, 10, N)         # mg m^-3 (total phytoplankton)
pp = rng.uniform(50, 1500, N)           # mg C m^-2 d^-1 (ballpark)
runoff = rng.exponential(0.1, N)        # arbitrary units
rono3 = rng.exponential(0.05, N)        # mmol N m^-2 d^-1 equiv (toy)
ronh4 = rng.exponential(0.03, N)
roo = rng.exponential(0.02, N)
rop = rng.exponential(0.01, N)
rosio2 = rng.exponential(0.04, N)
bathy = rng.uniform(10, 300, N)         # m (coastal to shelf)
month = rng.integers(1, 13, N)
day = rng.integers(1, 29, N)            # keep simple (28 days)

X = pd.DataFrame({
    'lon': lon, 'lat': lat, 'WCDepth': WCDepth, 'chl_interp': chl, 'pft_interp': pft,
    'sstost_interp': sstost, 'temp_interp': temp, 'd2m_interp': d2m, 't2m_interp': t2m,
    'sp_interp': sp, 'tp_interp': tp, 'u10_interp': u10, 'v10_interp': v10,
    'ssrdc_interp': ssrdc, 'phyc_interp': phyc, 'pp_interp': pp,
    'closest_rorunoff': runoff, 'closest_rono3': rono3, 'closest_ronh4': ronh4,
    'closest_roo': roo, 'closest_rop': rop, 'closest_rosio2': rosio2,
    'bathy_interp': bathy, 'month': month, 'day': day
})[feature_names]

# Create a synthetic "true" mapping to NO3_obs with physically-inspired correlations:
# - Increase with runoff, rono3, ronh4, bathymetry shallower (inverse)
# - Seasonal modulation via month, effect of temperature/pp/chl
true_no3 = (
    0.15*runoff + 0.8*rono3 + 0.5*ronh4
    + 0.002*(chl + phyc) - 0.03*(sstost-12).clip(min=-5, max=10)
    + 0.0005*pp
    + 0.002*(u10**2 + v10**2)
    + 0.01*(14 - (bathy/50))              # shallower => higher
    + 0.2*np.sin(2*np.pi*(month/12))
)

# Add heteroscedastic noise (higher variance with higher pp/chl)
noise = rng.normal(0, 0.2 + 0.0002*pp + 0.005*chl, N)
no3_obs = np.clip(true_no3 + noise, 0, None)  # prevent negatives

# A simple "model no3" proxy to emulate a biased baseline (CMEMS/ERSEM-like)
no3_model = np.clip(true_no3 * 0.85 + rng.normal(0, 0.3, N), 0, None)

# Provide the dataframe in same structure you'd expect during training
df = X.copy()
df['no3_obs'] = no3_obs
df['no3_model'] = no3_model

df.head()


In [ ]:

from sklearn.metrics import mean_squared_error
import numpy as np

def rmse(y_true, y_pred):
    return float(np.sqrt(mean_squared_error(y_true, y_pred)))

def bias(y_true, y_pred):
    return float(np.mean(y_pred - y_true))

def corr(y_true, y_pred):
    # Pearson correlation
    yt = np.asarray(y_true).ravel()
    yp = np.asarray(y_pred).ravel()
    if yt.std() == 0 or yp.std() == 0:
        return float('nan')
    return float(np.corrcoef(yt, yp)[0,1])


In [ ]:

import matplotlib.pyplot as plt

def plot_parity(y_true, y_pred, title):
    plt.figure(figsize=(6,6))
    plt.scatter(y_true, y_pred, s=6, alpha=0.5)
    minv = min(np.min(y_true), np.min(y_pred))
    maxv = max(np.max(y_true), np.max(y_pred))
    plt.plot([minv, maxv], [minv, maxv], lw=1)
    plt.xlabel("Observed NO$_3$ [mmol N m$^{-3}$]")
    plt.ylabel("Predicted NO$_3$ [mmol N m$^{-3}$]")
    plt.title(title)
    plt.tight_layout()
    plt.show()


In [ ]:

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.callbacks import EarlyStopping

features = df.drop(columns=['no3_obs','no3_model'])
target = df['no3_obs']

X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=11)

# -------- Baseline (for comparison) --------
lr = LinearRegression()
lr.fit(X_train, y_train)
y_lr = lr.predict(X_test)

# -------- Tiny Keras Emulator --------
model = tf.keras.Sequential([
    layers.Input(shape=(X_train.shape[1],)),
    layers.Dense(64, activation='relu'),
    layers.Dense(64, activation='relu'),
    layers.Dense(1)
])

model.compile(optimizer='adam', loss='mse')

es = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True, min_delta=1e-4)

history = model.fit(
    X_train, y_train,
    validation_split=0.2,
    epochs=100,
    batch_size=128,
    callbacks=[es],
    verbose=0
)

y_ml = model.predict(X_test, verbose=0).ravel()

print("Linear Regression baseline:")
print(f"  RMSE = {rmse(y_test, y_lr):.3f} mmol N m^-3")
print(f"  Bias = {bias(y_test, y_lr):.3f} mmol N m^-3")
print(f"  Corr = {corr(y_test, y_lr):.3f}")

print("\nTiny Keras emulator:")
print(f"  RMSE = {rmse(y_test, y_ml):.3f} mmol N m^-3")
print(f"  Bias = {bias(y_test, y_ml):.3f} mmol N m^-3")
print(f"  Corr = {corr(y_test, y_ml):.3f}")

plot_parity(y_test, y_lr, title="Baseline parity: Linear Regression (test set)")
plot_parity(y_test, y_ml, title="Emulator parity: Tiny Keras MLP (test set)")
